<a href="https://colab.research.google.com/github/phisan-chula/GNSS-Calibration/blob/main/GNSS_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pymap3d
!pip install pygeodesy
!pip install ipdb
!apt-get -y install geographiclib-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
geographiclib-tools is already the newest version (1.52-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [8]:
'''
GenRefBL.py : generates ia reference geodetic baseline
between two points (P1 and P2) with a specified length (k kilometers)
and forward azimuth (az21).

This straight-line baseline is an approximation of the true geodesic line,
which is the shortest path between two points on an ellipsoid
The script acknowledges that the generated straight-line baseline
(chord) will have a small difference compared to the true geodesic line
(s12) projected onto the ellipsoid surface.
'''

import os
import subprocess
import pymap3d as pm
import numpy as np
import pandas as pd
from scipy.optimize import fmin,minimize
from pyproj import Geod
from pygeodesy import dms

def AzTrunc180( az ):
    ''' truncating azimuth [-180...+180] degree '''
    _, az360 = divmod(az, 360) # round-off to one-circle
    div,mod = divmod( az360, 180 )
    if div>=1: return mod-180
    else: return mod

class BaseLine:
    def __init__(self):
        self.g = Geod(ellps='WGS84')
        self.TABLE  = list()
        self.TABLE_ = list()
        if 0:  # WatSutat
            self.SCENE = 'Flat-baseline Wat Sutat, Bangkok'
            self.lat2, self.lon2 = [ 13.75, 100.50  ]  # fixed !
            self.faz2 = 45.000
            self.BL_hae = [ -32 , -30]  # meter
        else:  # Intanon
            self.SCENE = 'Slopy-baseline Doi Intanont, Chiangmai'
            self.lat2, self.lon2 = [ 18.60, 98.50  ]  # fixed !
            self.faz2 = -135.000
            #self.BL_hae = [ 260 , 2526]  # meter  !!! DeltaH >> S12
            self.BL_hae = [ 260 , 1393]  # meter  !!! half-way to DOI
        assert -90  <=self.lat2 <=+90
        assert -180 <=self.lon2 <=+180
        assert -180 <=self.faz2 <=+180

    def PJ4_Inverse( self, lat1, lon1, lat2, lon2 ):
        ''' return forward azimuth at starting and terminus, -180 < az <+180 '''
        fwd_az1, bwd_az2, s12  = self.g.inv(lon1, lat1, lon2, lat2, radians=False )
        return fwd_az1,AzTrunc180(bwd_az2+180),s12

    def PJ4_Direct( self, lat1, lon1, az1, s12 ):
        ''' return lon2,lat2 of the terminus, also fwd_az2 at the terminus'''
        ''' PJ4_fwd() gives back azimuths of terminus!!!, reverse to forward'''
        lon2,lat2,bwd_az2  = self.g.fwd(lon1, lat1, az1, s12, radians=False )
        return lat2,lon2,AzTrunc180( bwd_az2+180 )

    def PrintBaseLine(self):
        print( f'Reference : {self.g}' )
        print( f'Target to P2:  phi lam  faz' )
        print( f'{self.lat2:.9f} {self.lon2:.9f}, {self.faz2:.9f} ' )
        print( f'{dms.toDMS(self.lat2,prec=5)}  {dms.toDMS(self.lon2,prec=5)} '\
               f'{dms.toDMS(self.faz2,prec=5)}' )
        print( f'Elevation P1/P2 hae : {self.BL_hae[0]:.3f} / {self.BL_hae[1]:.3f} meter' )
        COLS = [ 'baseline','s12', 'NSWm',
                 'P1_lat','P1_lng','P1_hae','P2_lat','P2_lng','P2_hae', 'faz1', 'faz2' ]
        self.dfTABLE = pd.DataFrame( self.TABLE, columns=COLS )
        FMT = ( '.3f','.3f','.0f','.9f','.9f','.3f','.9f','.9f','.3f', '.7f', '.7f')
        print( self.dfTABLE.to_markdown( floatfmt=FMT, index=False ) )
        self.dfTABLE_ = pd.DataFrame( self.TABLE_, columns=COLS )
        FMT_ = ( '.3f','.3f','.0f',None,None,'.3f',None,None,'.3f',None,None)
        print( self.dfTABLE_.to_markdown( floatfmt=FMT_ , index=False ) )
        self.dfTABLE_.to_pickle( "./TableBaseline.pkl" )

    def TableAppend(self, lat1, lon1 ):
        az1, az2, s12 = self.PJ4_Inverse( lat1,lon1, self.lat2, self.lon2 )
        XYZ1 = pm.geodetic2ecef( lat1,      lon1,      self.BL_hae[0], deg=True)
        XYZ2 = pm.geodetic2ecef( self.lat2, self.lon2, self.BL_hae[1], deg=True)
        bl = np.linalg.norm( np.array(XYZ2)-np.array(XYZ1) )
        if bl<5_000: min_obs = 20 # minutes
        else: min_obs = (2*bl/1_000)+10   # minutes
        self.TABLE.append( [  f'{bl:,.3f}' , f'{s12:,.3f}',  f'{min_obs:,.0f}',
                 f'{lat1:.9f}',     f'{lon1:.9f}',     f'{self.BL_hae[0]:.3f}',
                 f'{self.lat2:.9f}',f'{self.lon2:.9f}',f'{self.BL_hae[1]:.3f}',
                 f'{az1:.9f}',      f'{az2:.9f}'   ] )
        self.TABLE_.append( [ f'{bl:,.3f}' , f'{s12:,.3f}',  f'{min_obs:,.0f}',
          f'{dms.toDMS(lat1,prec=5)}', f'{dms.toDMS(lon1,prec=5)}', f'{self.BL_hae[0]:.3f}',
          f'{dms.toDMS(self.lat2,prec=5)}',f'{dms.toDMS(self.lon2,prec=5)}',f'{self.BL_hae[1]:.3f}',
          f'{dms.toDMS(az1,prec=2)}',f'{dms.toDMS(az2,prec=2)}'   ] )

    def Solve_Baseline(self, BL ):
        """ solve for baseline of length 'BL' , give terminus point lat2,lon2
            and its forward azimuth 'fwd_az2. Height above ellipsoid 'hae' of
            both ends are constrainted.
            returns: lat1,lon1 of the starting
        """
        def ObjFunc(X0,BL):
            lat1,lon1,hae1 = *X0, self.BL_hae[0]
            lat2,lon2,hae2 = self.lat2,self.lon2, self.BL_hae[1]
            faz1_,faz2_,s12_  = self.PJ4_Inverse(lat1,lon1,lat2,lon2 )
            XYZ1 = pm.geodetic2ecef( lat1, lon1, hae1 , deg=True)
            XYZ2 = pm.geodetic2ecef( lat2, lon2, hae2 , deg=True)
            bl = np.linalg.norm( np.array(XYZ2)-np.array(XYZ1) )
            diff_az2 = AzTrunc180( faz2_-self.faz2 )
            L2norm = np.linalg.norm( np.array([bl-BL ,diff_az2] ) )
            return L2norm
        lat1_,lon1_,_ = self.PJ4_Direct(self.lat2, self.lon2, AzTrunc180(180+self.faz2), BL)
        lat1,lon1 = fmin( ObjFunc, np.array([lat1_,lon1_]), args=(BL,),
                    xtol=1E-4, ftol=1E-9, maxiter=1000, disp=True )
        CHK = '''GeodSolve -i --input-string "{} {} {} {}"'''.format(
                                lat1, lon1, self.lat2, self.lon2 )
        print( CHK )
        res = subprocess.run( CHK, shell=True, capture_output=True )
        print(res.stdout.decode('ascii'))
        return lat1, lon1


In [11]:
##################################################
ref = BaseLine()
for bl in [ 2,5,10,20,30,40,50,60,70,80]:
    print(f'=================== baseline length = {bl:} km  ==================')
    lat1,lon1 = ref.Solve_Baseline( bl*1_000 )  # a bit neater ....
    ref.TableAppend( lat1, lon1 )

print('========================== summary ============================')
print(f'Scenario : {ref.SCENE:}...')
ref.PrintBaseLine()
#import pdb; pdb.set_trace()

=================== baseline length = 2 km  ==================
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 125
         Function evaluations: 241
GeodSolve -i --input-string "18.610527053085878 98.51104138732823 18.6 98.5"
-134.99647728 -135.00000000 1647.909

=================== baseline length = 5 km  ==================
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 115
         Function evaluations: 226
GeodSolve -i --input-string "18.631103843194158 98.53262944572009 18.6 98.5"
-134.98958414 -135.00000000 4869.307

=================== baseline length = 10 km  ==================
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 103
         Function evaluations: 200
GeodSolve -i --input-string "18.663451678492645 98.5665829150563 18.6 98.5"
-134.97872781 -135.00000000 9934.319

=================== baseline length = 20 km  =

In [10]:
! GeodSolve -i --input-string "13.430217230673703 100.17352620734948 13.75 100.5"

44.92328655 45.00000000 50000.373
